In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16, VGG16_Weights
import matplotlib.pyplot as plt
import numpy as np
import time


In [3]:

# --- CNN from Scratch ---
print("\n--- Training CNN from Scratch ---")
cnn_model = CustomCNN()
optimizer_cnn = optim.Adam(cnn_model.parameters(), lr=LEARNING_RATE)
history_cnn = train_model(cnn_model, trainloader, criterion, optimizer_cnn, epochs=EPOCHS)
final_loss_cnn, final_acc_cnn = evaluate_model(cnn_model, testloader, criterion)



--- Training CNN from Scratch ---
Epoch 1/10 | Time: 14.58s | Loss: 1.5530 | Acc: 43.35% | Val Loss: 1.1924 | Val Acc: 56.94%
Epoch 2/10 | Time: 18.09s | Loss: 1.1701 | Acc: 58.67% | Val Loss: 0.9497 | Val Acc: 66.19%
Epoch 3/10 | Time: 16.48s | Loss: 1.0043 | Acc: 64.84% | Val Loss: 0.8740 | Val Acc: 69.34%
Epoch 4/10 | Time: 13.70s | Loss: 0.8889 | Acc: 69.26% | Val Loss: 0.8280 | Val Acc: 71.11%
Epoch 5/10 | Time: 11.36s | Loss: 0.8048 | Acc: 72.10% | Val Loss: 0.7675 | Val Acc: 73.91%
Epoch 6/10 | Time: 19.18s | Loss: 0.7378 | Acc: 74.16% | Val Loss: 0.7692 | Val Acc: 74.14%
Epoch 7/10 | Time: 13.63s | Loss: 0.6863 | Acc: 76.07% | Val Loss: 0.7541 | Val Acc: 75.10%
Epoch 8/10 | Time: 15.05s | Loss: 0.6345 | Acc: 77.55% | Val Loss: 0.7470 | Val Acc: 74.92%
Epoch 9/10 | Time: 16.52s | Loss: 0.5926 | Acc: 79.23% | Val Loss: 0.7142 | Val Acc: 75.91%
Epoch 10/10 | Time: 15.25s | Loss: 0.5555 | Acc: 80.43% | Val Loss: 0.7566 | Val Acc: 75.35%
Finished Training


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

# ==============================================================================
# SECTION 1: SETUP AND HYPERPARAMETERS RESNET-18
# ==============================================================================
print("--- Setting up hyperparameters and device ---")
# Set the device to a GPU if available, otherwise use the CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define hyperparameters
# NOTE: Training from scratch requires more epochs to converge.
# 40-50 epochs is a good starting point.
EPOCHS = 40
BATCH_SIZE = 128
LEARNING_RATE = 0.01 # Initial learning rate for SGD
NUM_CLASSES = 10     # For CIFAR-10


# ==============================================================================
# SECTION 2: DATA LOADING AND AUGMENTATION
# ==============================================================================
print("\n--- Preparing CIFAR-10 dataset ---")
# Define data augmentation and normalization for the training set
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Define normalization for the test set (no augmentation)
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load the datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# Create the data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


# ==============================================================================
# SECTION 3: MODEL AND HELPER FUNCTIONS
# ==============================================================================

def create_resnet18_from_scratch(num_classes=10):
    """
    Creates a ResNet-18 model with random initial weights (trained from scratch).
    """
    print("Initializing ResNet-18 model with random weights.")
    # Load a ResNet18 model with weights=None for random initialization
    model = resnet18(weights=None, num_classes=num_classes)
    
    # --- MODIFICATION FOR CIFAR-10 ---
    # To improve performance on small 32x32 images, it's highly recommended
    # to make the first layer less aggressive.
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    # --- END OF MODIFICATION ---
    
    return model

def train_model(model, trainloader, criterion, optimizer, scheduler=None, epochs=20, device='cpu'):
    """Function to train a PyTorch model."""
    history = {'loss': [], 'accuracy': []}
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        print(f"\n--- Epoch {epoch+1}/{epochs} ---")
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
        if scheduler:
            scheduler.step()
        epoch_loss = running_loss / total_samples
        epoch_acc = (correct_predictions / total_samples) * 100
        history['loss'].append(epoch_loss)
        history['accuracy'].append(epoch_acc)
        print(f"Epoch Summary | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")
    print('\n--- Finished Training ---')
    return history

def evaluate_model(model, testloader, criterion, device='cpu'):
    """Function to evaluate a PyTorch model's performance."""
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    final_loss = running_loss / total_samples
    final_acc = (correct_predictions / total_samples) * 100
    return final_loss, final_acc


# ==============================================================================
# SECTION 4: MAIN EXECUTION BLOCK
# ==============================================================================

print("\n--- Initializing Model, Loss, and Optimizer ---")
# Create the model FROM SCRATCH and move it to the configured device
model_from_scratch = create_resnet18_from_scratch(num_classes=NUM_CLASSES)
model_from_scratch.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer to train ALL model parameters
optimizer = optim.SGD(model_from_scratch.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=5e-4)

# Define a learning rate scheduler
scheduler = StepLR(optimizer, step_size=15, gamma=0.1) # Adjusted step_size for more epochs

# --- Start Training ---
print("\n--- Training ResNet-18 from scratch ---")
history = train_model(model_from_scratch, trainloader, criterion, optimizer, scheduler=scheduler, epochs=EPOCHS, device=device)

# --- Evaluate the Model ---
print("\n--- Evaluating model on the test set ---")
final_loss, final_acc = evaluate_model(model_from_scratch, testloader, criterion, device=device)

print(f"\nFinal Test Results:")
print(f"Loss: {final_loss:.4f}")
print(f"Accuracy: {final_acc:.2f}%")

--- Setting up hyperparameters and device ---
Using device: cuda:0

--- Preparing CIFAR-10 dataset ---
Files already downloaded and verified
Files already downloaded and verified

--- Initializing Model, Loss, and Optimizer ---
Initializing ResNet-18 model with random weights.

--- Training ResNet-18 from scratch ---

--- Epoch 1/40 ---
Epoch Summary | Loss: 1.5426 | Accuracy: 43.01%

--- Epoch 2/40 ---
Epoch Summary | Loss: 1.1003 | Accuracy: 60.48%

--- Epoch 3/40 ---
Epoch Summary | Loss: 0.8588 | Accuracy: 69.54%

--- Epoch 4/40 ---
Epoch Summary | Loss: 0.7161 | Accuracy: 75.09%

--- Epoch 5/40 ---
Epoch Summary | Loss: 0.6250 | Accuracy: 78.16%

--- Epoch 6/40 ---
Epoch Summary | Loss: 0.5647 | Accuracy: 80.39%

--- Epoch 7/40 ---
Epoch Summary | Loss: 0.5124 | Accuracy: 82.37%

--- Epoch 8/40 ---
Epoch Summary | Loss: 0.4658 | Accuracy: 83.81%

--- Epoch 9/40 ---
Epoch Summary | Loss: 0.4323 | Accuracy: 85.02%

--- Epoch 10/40 ---
Epoch Summary | Loss: 0.4018 | Accuracy: 85.94%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16, VGG16_Weights

# ==============================================================================
# SECTION 1: SETUP AND HYPERPARAMETERS
# ==============================================================================
print("--- Setting up hyperparameters and device ---")
# Set the device to a GPU if available, otherwise use the CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define hyperparameters
EPOCHS = 20  # Epochs for fine-tuning the full model
BATCH_SIZE = 128
# Use a very low learning rate for fine-tuning the entire network
LEARNING_RATE_FINETUNE = 0.0001 
NUM_CLASSES = 10                  # For CIFAR-10


# ==============================================================================
# SECTION 2: DATA LOADING AND AUGMENTATION
# ==============================================================================
print("\n--- Preparing CIFAR-10 dataset ---")
# VGG models work best with larger images, so we resize CIFAR-10.
IMAGE_SIZE = 64

# Define data augmentation and normalization for the training set
train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomCrop(IMAGE_SIZE, padding=8),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Define normalization for the test set (no augmentation)
test_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load the datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# Create the data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


# ==============================================================================
# SECTION 3: MODEL AND HELPER FUNCTIONS
# ==============================================================================

def create_vgg16_finetune_model(num_classes=10):
    """Creates a pre-trained VGG-16 model where ALL layers are trainable."""
    # Load a pre-trained VGG-16 model
    model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
    
    # NOTE: The layer freezing loop is removed. 
    # All layers (features and classifier) will be trainable.
        
    # Replace the final layer of the classifier
    num_ftrs = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(num_ftrs, num_classes)
    
    return model

def train_model(model, trainloader, criterion, optimizer, epochs=10, device='cpu'):
    """Function to train a PyTorch model."""
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        print(f"\n--- Epoch {epoch+1}/{epochs} ---")
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        epoch_loss = running_loss / total_samples
        epoch_acc = (correct_predictions / total_samples) * 100
        print(f"Epoch Summary | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")
    print('\n--- Finished Training ---')

def evaluate_model(model, testloader, criterion, device='cpu'):
    """Function to evaluate a PyTorch model's performance."""
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    final_loss = running_loss / total_samples
    final_acc = (correct_predictions / total_samples) * 100
    print(f"\nFinal Test Results | Loss: {final_loss:.4f} | Accuracy: {final_acc:.2f}%")
    return final_loss, final_acc


# ==============================================================================
# SECTION 4: MAIN EXECUTION BLOCK
# ==============================================================================

print("\n--- Initializing VGG-16 Model for Fine-Tuning ---")
# Create the VGG-16 model with all layers trainable
finetune_model = create_vgg16_finetune_model(num_classes=NUM_CLASSES)
finetune_model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer to train ALL parameters of the model
optimizer = optim.Adam(finetune_model.parameters(), lr=LEARNING_RATE_FINETUNE)

# --- Start Training ---
print("\n--- Fine-Tuning the Full VGG-16 Model ---")
train_model(finetune_model, trainloader, criterion, optimizer, epochs=EPOCHS, device=device)

# --- Evaluate the Model ---
print("\n--- Evaluating fine-tuned VGG-16 model on the test set ---")
evaluate_model(finetune_model, testloader, criterion, device=device)

--- Setting up hyperparameters and device ---
Using device: cuda:0

--- Preparing CIFAR-10 dataset ---
Files already downloaded and verified
Files already downloaded and verified

--- Initializing VGG-16 Model for Fine-Tuning ---

--- Fine-Tuning the Full VGG-16 Model ---

--- Epoch 1/20 ---
